In [ ]:
pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np
import category_encoders as ce
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
data = pd.read_csv('./drive/MyDrive/dataset.csv')

In [ ]:
# encoder1 = ce.HashingEncoder(cols='saddr',n_components=5)
# data = encoder1.fit_transform(data)
# encoder2 = ce.HashingEncoder(cols='daddr',n_components=5)
# data = encoder2.fit_transform(data)

In [ ]:
len(data)

733705

In [ ]:
data.dtypes

pkSeqID                int64
proto                 object
saddr                 object
sport                 object
daddr                 object
dport                 object
seq                    int64
stddev               float64
N_IN_Conn_P_SrcIP      int64
min                  float64
state_number           int64
mean                 float64
N_IN_Conn_P_DstIP      int64
drate                float64
srate                float64
max                  float64
attack                 int64
category              object
subcategory           object
dtype: object

In [ ]:
df = data.drop(['pkSeqID','subcategory','attack','dport','sport'],axis=1)

In [ ]:
#encoder1 = ce.HashingEncoder(cols='saddr',n_components=6)

encoder1 = ce.BinaryEncoder(cols=['saddr'],return_df=True)
df = encoder1.fit_transform(df)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
encoder2 = ce.BinaryEncoder(cols=['daddr'],return_df=True)
df = encoder2.fit_transform(df)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
proto_encoded = pd.get_dummies(data=df['proto'],drop_first=True)

In [ ]:
df = pd.concat([df,proto_encoded],axis=1)
df.drop('proto',axis=1,inplace=True)

In [ ]:
df.dtypes

saddr_0                int64
saddr_1                int64
saddr_2                int64
saddr_3                int64
saddr_4                int64
daddr_0                int64
daddr_1                int64
daddr_2                int64
daddr_3                int64
daddr_4                int64
daddr_5                int64
daddr_6                int64
seq                    int64
stddev               float64
N_IN_Conn_P_SrcIP      int64
min                  float64
state_number           int64
mean                 float64
N_IN_Conn_P_DstIP      int64
drate                float64
srate                float64
max                  float64
category              object
icmp                   uint8
ipv6-icmp              uint8
tcp                    uint8
udp                    uint8
dtype: object

In [ ]:
scaler = StandardScaler()
X = df.drop('category',axis=1)
y = df['category']
X = scaler.fit_transform(X)

In [ ]:

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=109)

In [ ]:
clf = LogisticRegression(random_state=0,multi_class='multinomial').fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
predictions = clf.predict(X_test)
print(accuracy_score(y_test,predictions))

0.9853347053652354


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = df['category'].values
y = ohe.fit_transform(y.reshape(-1,1)).toarray()
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=109)

In [ ]:
model = Sequential()
model.add(Dense(16,input_dim=26,activation='relu'))
model.add(Dense(12,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=64)

Epoch 1/100
9172/9172 [==============================] - 11s 1ms/step - loss: 0.1766 - accuracy: 0.9397
Epoch 2/100
9172/9172 [==============================] - 10s 1ms/step - loss: 0.0262 - accuracy: 0.9903
Epoch 3/100
9172/9172 [==============================] - 10s 1ms/step - loss: 0.0210 - accuracy: 0.9924
Epoch 4/100
9172/9172 [==============================] - 10s 1ms/step - loss: 0.0181 - accuracy: 0.9933
Epoch 5/100
9172/9172 [==============================] - 10s 1ms/step - loss: 0.0160 - accuracy: 0.9941
Epoch 6/100
9172/9172 [==============================] - 10s 1ms/step - loss: 0.0148 - accuracy: 0.9947
Epoch 7/100
9172/9172 [==============================] - 10s 1ms/step - loss: 0.0135 - accuracy: 0.9951
Epoch 8/100
9172/9172 [==============================] - 10s 1ms/step - loss: 0.0132 - accuracy: 0.9952
Epoch 9/100
9172/9172 [==============================] - 10s 1ms/step - loss: 0.0127 - accuracy: 0.9953
Epoch 10/100
9172/9172 [==============================] - 10s 1m

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

In [ ]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

Accuracy is: 99.40371130086343
